<a href="https://colab.research.google.com/github/garciafranciscomartn/nextflow_intro_rsg/blob/main/Modulo1_Introducci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a Nextflow

Comenzamos ejecutando FASTQC directamente en la terminal usando un archivo de prueba. Para eso vamos a crear la carpeta en la que vamos a trabajar y luego realizamos la descarga:
```bash
# Creamos la carpeta de trabajo
mkdir Nextflow_RSG
# Nos movemos a esa carpeta
cd Nextflow_RSG
# Descargamos un archivo FASTQ de prueba pequeño

wget -c https://raw.githubusercontent.com/nf-core/test-datasets/modules/data/genomics/homo_sapiens/illumina/fastq/test_1.fastq.gz

# Ejecutamos FASTQC

fastqc test_1.fastq.gz
```
Esto genera un reporte de control de calidad para el archivo FASTQ.

Verificamos los archivos generados
```
ls test_1_fastqc*
```
Deberíamos ver dos archivos:

-   test_1_fastqc.html - Reporte visual
-   test_1_fastqc.zip - Datos del reporte

Podemos abrir el reporte HTML en un navegador si deseamos

## ¿Cómo se vería esto escrito como un workflow de Nextflow?

En la carpeta donde estamos trabajando tenemos un script completamente funcional llamado hola_fastqc.nf que hace lo mismo que antes (ejecutar FASTQC) pero con Nextflow.

Abramos el script hola_fastqc.nf en el editor para analizar la estructura:

```
#!/usr/bin/env nextflow

/*
* Usar FASTQC para generar un reporte de control de calidad
*/

process runFastQC {
    container "community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8"

    
    input:
    path fastq_file
    
    output:
    path "*.html"
    path "*.zip"

    script:
    """
    fastqc ${fastq_file}
    """
}   

workflow {
    // Creamos canal con archivo input
    input_ch = Channel.fromPath("test_1.fastq.gz")
    
    // Ejecutamos FastQC
    runFastQC(input_ch)
}
```

Como podemos ver, un script de Nextflow involucra dos tipos principales de componentes: uno o más procesos, y el workflow en sí.

Cada proceso describe qué operación(es) debe realizar el paso correspondiente en el pipeline, mientras que el workflow describe la lógica de flujo de datos que conecta los pasos.

El primer bloque de código describe un proceso.
```
/*
* Usar FASTQC para generar un reporte de control de calidad
*/
process runFastQC {
    container "community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8"
    
    input:
    path fastq_file
    
    output:
    path "*.html"
    path "*.zip"

    script:
    """
    fastqc ${fastq_file}
    """
}
```
Esta es una definición de proceso muy mínima que contiene:

-   Una directiva con el *container* para especificar el software necesario (sobre esta parte profundizaremos más adelante cuando hablemos de containers).
-   Una directiva que indica cual es el input
-   Una directiva que indica cuales son los output esperados
-   El script a ejecutar, donde se indica entre las tres comillas(""") el comando a ejecutar

El segundo bloque de código describe el workflow en sí.
```
workflow {
    // Creamos canal con archivo input
    input_ch = Channel.fromPath("test_1.fastq.gz")
    
    // Ejecutamos FastQC
    runFastQC(input_ch)
}
```
Esta es una definición de workflow muy mínima que consiste en una llamada al proceso runFastQC.

Ahora vamos a correr el script, monitorear la ejecución y encontrar los outputs.

En la terminal, ejecutamos el siguiente comando:

```
nextflow run hola_fastqc.nf
```

La salida de consola debería verse algo así:

```
Launching `hola_fastqc.nf` [determined_liskov] DSL2 - revision: 76d41ecada

executor >  local (1)
[ee/c9ad53] runFastQC (1) | 1 of 1 ✔
```

¡Felicidades, acabamos de ejecutar nuestro primer script de Nextflow con FASTQC!

La línea más importante aquí es la última, esto nos dice que el proceso runFastQC fue ejecutado exitosamente una vez (1 of 1 ✔).

Cuando ejecutamos Nextflow por primera vez en el directorio donde estamos ubicados, crea un directorio llamado work donde escribirá todos los archivos generados durante la ejecución. Dentro del directorio work, Nextflow organiza las salidas y los registros por cada llamada a un proceso creando un subdirectorio anidado, nombrado con un hash para hacerlo único, donde colocará todos los archivos de entrada necesarios (usando enlaces simbólicos por defecto), escribirá archivos auxiliares, y guardará los registros y cualquier salida del proceso.

La ruta a ese subdirectorio se muestra en forma truncada entre corchetes en la salida de la consola. Observando lo que obtuvimos en la ejecución mostrada anteriormente, la línea del registro de consola para el proceso runFastQC comienza con [ee/c9ad53]. Eso corresponde a la siguiente ruta de directorio: work/ee/c9ad536657079ac416c2608c40c150.

Veamos las carpetas y archivos creados al correr el script:
```
tree -a work
```
Deberías verse algo así:
```
work/
└── ee
    └── c9ad536657079ac416c2608c40c150
        ├── .command.begin
        ├── .command.err
        ├── .command.log
        ├── .command.out
        ├── .command.run
        ├── .command.sh
        ├── .exitcode
        ├── test_1.fastq.gz -> /home/fg47909/nextflow_rsg/test_1.fastq.gz
        ├── test_1_fastqc.html
        └── test_1_fastqc.zip
```
Contenido en la ultima carpeta varios archivos entre los cuales se encuentra nuestro output deseado. Veamos con más detalle los archivos:

*   .command.begin: Metadatos relacionados con el inicio de la ejecución de la llamada al proceso.
*   .command.err: Mensajes de error (stderr) emitidos por la llamada al proceso.
*   .command.log: Registro completo de salida generado por la llamada al proceso.
*   .command.out: Salida estándar (stdout) generada por la llamada al proceso.
*   .command.run: Script completo que Nextflow ejecutó para realizar la llamada al proceso.
*   .command.sh: El comando que realmente fue ejecutado por la llamada al proceso.
*   .exitcode: El código de salida resultante del comando.

El archivo .command.sh es especialmente útil porque te muestra qué comando ejecutó realmente Nextflow.

Todos estos serían archivos auxiliares y de registro del script, los archivos de output reales del proceso runFastQC son:

*   test_1_fastqc.html - Reporte de control de calidad
*   test_1_fastqc.zip - Datos del reporte

Además tenemos el enlace simbólico hacia nuestro archivo input (el fastq)
*   test_1.fastq.gz -> /home/fg47909/nextflow_rsg/test_1.fastq.gz
Donde vemos con la flecha cual seria la ubicacion del archivo

Como vemos en este script, hay varios elementos que se definieron y vamos a desglosarlo elemento por elemento:

1. Shebang y Comentarios

```
#!/usr/bin/env nextflow
/*
* Usar FASTQC para generar un reporte de control de calidad
*/
```

*   Shebang (#!/usr/bin/env nextflow): Le dice al sistema que este archivo debe ejecutarse con Nextflow
*   Comentario de bloque(Comienza con /* y concluye con */): Describe brevemente qué hace el script
2. Definición del Proceso

```
process runFastQC {
    container "community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8"
   
    input:
    path fastq_file
    
    output:
    path "*.html"
    path "*.zip"

    script:
    """
    fastqc ${fastq_file}
    """
}
```

Elementos del proceso:
-   process runFastQC: Define un proceso llamado runFastQC
-   container "community.wave.seqera.io/library/fastp_fastqc_multiqc:46d8231a252ab2c8": Especifica el container de docker con el software que vamos a usar.
-   input: Define qué datos recibe el proceso.
-   path fastq_file: Recibe un archivo (cualquier tipo de archivo) y lo llama fastq_file.
-   output: Define qué archivos produce el proceso.
-   path "*.html": Todos los archivos HTML generados.
-   path "*.zip": Todos los archivos ZIP generados.
-   script: Contiene el comando que se ejecutará.
-   fastqc ${fastq_file}: Ejecuta FastQC sobre el archivo de entrada.


3. Workflow Principal

```
workflow {
    // Creamos canal con archivo input
    input_ch = Channel.fromPath("test_1.fastq.gz")
    
    // Ejecutamos FastQC
    runFastQC(input_ch)
}
```

Elementos del workflow:
-   workflow {}: Define el flujo de trabajo principal
-   input_ch = Channel.fromPath("test_1.fastq.gz"):
      Crea un canal llamado input_ch
      Channel.fromPath() busca archivos que coincidan con el patrón, en este caso, busca específicamente el archivo "test_1.fastq.gz"
-   runFastQC(input_ch): Ejecuta el proceso runFastQC usando el canal como entrada

No se asusten por la cantidad de elementos nuevos, la idea va a ser explicar su uso a medida que avancemos.
